In [1]:
import sklearn as sk
import pandas as pd
import numpy as np
from os import getenv
from datetime import datetime
from amb_sdk.sdk import DarwinSdk
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
cycle_df = pd.read_csv('./data/Cleaned_Cycle_Trips_Weather_Dates.csv')
weather_df = pd.read_csv('./data/austin_weather.csv')

In [3]:
cycle_df.dtypes

Unnamed: 0                 int64
Trip ID                    int64
Membership Type           object
Bicycle ID               float64
Checkout Date             object
Checkout Time             object
Checkout Kiosk            object
Return Kiosk              object
Trip Duration Minutes      int64
Month                      int64
Year                       int64
dtype: object

In [7]:
cycle_df['Checkout Date'] = pd.to_datetime(
    cycle_df['Checkout Date'], infer_datetime_format=True
    ).dt.normalize() #strftime('%m/%d/%Y') # format='%m/%d/%Y').dt.date

cycle_df['Checkout Time'] = pd.to_timedelta(
    cycle_df['Checkout Time']) #, format='%H:%M:%S')\
#.dt.strftime('%H:%M') #This turns it back to a object dtype


weather_df['Date'] = pd.to_datetime(
    weather_df['Date'], infer_datetime_format=True
    ).dt.normalize() #strftime('%m/%d/%Y') # format='%m/%d/%Y').dt.date

In [8]:
cycle_df.dtypes

Unnamed: 0                         int64
Trip ID                            int64
Membership Type                   object
Bicycle ID                       float64
Checkout Date             datetime64[ns]
Checkout Time            timedelta64[ns]
Checkout Kiosk                    object
Return Kiosk                      object
Trip Duration Minutes              int64
Month                              int64
Year                               int64
dtype: object

In [9]:
print('Cycle Trips Earliest Data',cycle_df['Checkout Date'].dt.date.min())
print('Cycle Trips Latest Data',cycle_df['Checkout Date'].dt.date.max())

print('Weather Earliest Data',weather_df['Date'].dt.date.min())
print('Weather Latest Data',weather_df['Date'].dt.date.max())

Cycle Trips Earliest Data 2013-12-21
Cycle Trips Latest Data 2017-07-31
Weather Earliest Data 2013-12-21
Weather Latest Data 2017-07-31


In [17]:
cycle_df['Checkout WeekDay'] = cycle_df['Checkout Date'].dt.day_name()
cycle_df.drop('Unnamed: 0', axis=1, inplace=True)
weather_df['Weekday'] = weather_df['Date'].dt.day_name()


Index(['Trip ID', 'Membership Type', 'Bicycle ID', 'Checkout Date',
       'Checkout Time', 'Checkout Kiosk', 'Return Kiosk',
       'Trip Duration Minutes', 'Month', 'Year', 'Checkout WeekDay'],
      dtype='object')

In [18]:
cycle_df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Date,Checkout Time,Checkout Kiosk,Return Kiosk,Trip Duration Minutes,Month,Year,Checkout WeekDay
0,9900285854,Annual (San Antonio B-cycle),207.0,2014-10-26,13:12:00,West & 6th St.,Rainey St @ Cummings,76,10,2014,Sunday
1,9900285855,24-Hour Kiosk (Austin B-cycle),969.0,2014-10-26,13:12:00,Convention Center / 4th St. @ MetroRail,Pfluger Bridge @ W 2nd Street,58,10,2014,Sunday
2,9900285856,Annual Membership (Austin B-cycle),214.0,2014-10-26,13:12:00,West & 6th St.,8th & Congress,8,10,2014,Sunday
3,9900285857,24-Hour Kiosk (Austin B-cycle),745.0,2014-10-26,13:12:00,Zilker Park at Barton Springs & William Barton...,Zilker Park at Barton Springs & William Barton...,28,10,2014,Sunday
4,9900285858,24-Hour Kiosk (Austin B-cycle),164.0,2014-10-26,13:12:00,Bullock Museum @ Congress & MLK,Convention Center/ 3rd & Trinity,15,10,2014,Sunday
